<a href="https://colab.research.google.com/github/ronykroy/DNN-NLP-and-other-stuff/blob/master/H20_multiClass_newsGroups_AutoML_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# V1 Vs V2
# V1 had the issue that 'label was one of the predictor columns.. so data leaked that way...
# v2 removes label n train columns
# v3: remove the text too ?

In [0]:
!apt-get install default-jre
!java -version
# install Java.. because.. there is nothing in this colab VM

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


In [0]:
! pip install h2o
# H2O too to be installed SMH

In [0]:
import h2o

In [0]:
h2o.init(max_mem_size="4G")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 28 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.8
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_unknownUser_0ghsh3
H2O cluster total nodes:,1
H2O cluster free memory:,3.326 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [0]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
df["Train"] = np.random.rand(len(df)) < 0.8 # get a col full of train boolean 

In [0]:
df.Train.value_counts()

True     9053
False    2261
Name: Train, dtype: int64

In [0]:
reviews = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train = reviews[reviews["Train"] == "True"]
test = reviews[reviews["Train"] == "False"]

In [0]:
train.head(2)

label,text,Train
17,"Well i'm not sure about the story nad it did seem biased. What I disagree with is your statement that the U.S. Media is out to ruin Israels reputation. That is rediculous. The U.S. media is the most pro-israeli media in the world. Having lived in Europe I realize that incidences such as the one described in the letter have occured. The U.S. media as a whole seem to try to ignore them. The U.S. is subsidizing Israels existance and the Europeans are not (at least not to the same degree). So I think that might be a reason they report more clearly on the atrocities. What is a shame is that in Austria, daily reports of the inhuman acts commited by Israeli soldiers and the blessing received from the Government makes some of the Holocaust guilt go away. After all, look how the Jews are treating other races when they got power. It is unfortunate.",True
0,"Yeah, do you expect people to read the FAQ, etc. and actually accept hard atheism? No, you need a little leap of faith, Jimmy. Your logic runs out of steam! Jim, Sorry I can't pity you, Jim. And I'm sorry that you have these feelings of denial about the faith you need to get by. Oh well, just pretend that it will all end happily ever after anyway. Maybe if you start a new newsgroup, alt.atheist.hard, you won't be bummin' so much? Bye-Bye, Big Jim. Don't forget your Flintstone's Chewables! :) -- Bake Timmons, III",True


In [0]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [0]:
# TODO: review how effective this is in removing junk \r \n etc
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [0]:
%%time
# Break reviews into sequence of words
words = tokenize(reviews["text"]) # not sure if there is some sort of leakage here...

CPU times: user 4.25 s, sys: 74.3 ms, total: 4.33 s
Wall time: 16.7 s


In [0]:
words.head()

C1
well
sure
story
nad
seem
biased
disagree
statement
media
ruin


In [0]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words) # train on the words thing we just got

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [0]:
# Sanity check - find synonyms for the word 'president'
w2v_model.find_synonyms("president", count = 5) # not bad actually :)

OrderedDict([('yeltsin', 0.8460185527801514),
             ('mubarak', 0.8207446932792664),
             ('stephanopoulos', 0.811212956905365),
             ('miyazawa', 0.7764836549758911),
             ('dole', 0.7588897347450256)])

In [0]:
# Calculate a vector for each review
review_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [0]:
review_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.0388161,0.0433147,-0.0671115,-0.168748,0.135832,0.161679,0.0361269,0.119345,-0.0642259,0.0224799,0.000716771,-0.000478264,0.0584357,0.121972,-0.338503,-0.0317321,-0.258186,0.00356372,-0.029555,0.0928804,-0.0326285,0.105699,-0.195085,-0.0682508,0.0411102,0.209399,0.0859861,-0.0889125,-0.0159964,0.10744,-0.0561944,0.156689,0.0228213,-0.0302436,-0.0229776,-0.0101779,0.0340967,0.0853087,-0.242132,-0.25744,0.0603646,-0.0600112,0.108552,-0.0748304,0.0766511,0.00994602,0.0140275,0.0776616,0.00908043,-0.0110328,0.107063,-0.0711367,-0.117333,0.122291,-0.127887,0.048512,-0.0255488,-0.0388489,0.0359795,0.0423406,-0.142347,-0.0220931,0.10858,0.0204853,0.044563,-0.0338904,0.0589644,-0.106907,-0.0151874,-0.1017,0.0171318,-0.0053127,0.133991,0.0213216,-0.00739897,-0.00150762,-0.0113215,0.067952,-0.0634168,-0.00163656,0.0629755,0.0622206,-0.0223955,-0.0757768,0.0801227,0.203171,0.0369031,-0.0189166,0.165795,-0.0110198,-0.00439393,-0.0969211,-0.0126158,0.0514497,0.14552,0.113774,-0.0277334,0.132128,-0.0123805,0.17857
0.0902982,-0.0390844,-0.0458691,-0.167126,0.114107,0.0496239,-0.00843299,0.0898202,-0.0850967,-0.00947367,0.0973465,-0.116795,0.149379,0.154695,-0.240081,-0.0444662,-0.184735,-0.0327223,-0.121734,-0.0116894,-0.0199944,0.0938716,-0.171292,-0.0509831,0.0847219,0.0830923,0.0636284,-0.17799,0.0926193,0.0887863,-0.117524,-0.0384156,-0.0301427,-0.0148886,0.00800723,0.00293339,0.0638538,0.0424487,-0.11034,-0.131078,0.0814415,-0.00142114,0.14971,-0.0652322,0.0967615,-0.0215784,-0.0366854,-0.00876677,-0.0750503,-0.0194402,0.149285,-0.14217,-0.0981455,0.0719865,0.0792275,-0.035925,-0.00863693,-0.0916927,-0.00689619,-0.0307563,-0.196255,-0.0483425,-0.0560411,-0.00224286,-0.0231206,0.0751971,0.0744402,-0.109816,0.0535332,0.000621746,-0.128502,0.0114885,0.0629995,0.0811286,-0.153731,-0.0243036,-0.0723219,0.0633655,-0.0818336,-0.0533623,0.110762,-0.109733,-0.0164192,-0.0347722,0.107725,0.15412,-0.112623,0.0113255,0.142008,0.0248181,-0.112353,-0.0449256,0.0178335,0.0221197,0.0623327,0.0737877,-0.0204318,0.0824858,0.0671591,0.251788
0.0234295,0.0604782,-0.0279418,-0.208354,0.113017,0.174975,0.0671824,0.15635,-0.111997,0.0556891,0.0253962,-0.0568237,0.158468,0.119323,-0.257667,-0.113459,-0.272687,0.00932919,-0.0631726,0.0607729,-0.0719439,0.0754415,-0.113717,-0.016651,-0.00926196,0.189643,0.142804,-0.0643157,0.013161,0.11369,-0.0981279,0.172396,0.040464,0.00479784,0.0119593,-0.0188519,0.0242818,0.0281751,-0.169056,-0.271642,0.146099,0.00864597,0.0892739,-0.0824431,0.0386298,-0.0389177,-0.0393564,0.0275586,-0.019826,-0.0238678,0.0453329,-0.105981,-0.0721034,0.145189,-0.00770893,0.0532874,-0.00585818,-0.0898943,0.0397316,0.0211619,-0.142987,-0.0870881,0.0355105,-0.00441325,0.0882858,-0.0184847,0.00900559,-0.0506014,0.0270835,-0.0772397,-0.00854661,0.008197,0.0271407,0.0859015,-0.0196735,0.00309467,-0.061586,0.129375,-0.0838474,0.0135679,0.108902,0.0454641,-0.0407603,-0.0423955,0.0871443,0.175314,0.0477649,0.0244942,0.204134,-0.00446851,-0.0460091,-0.137266,0.0220153,0.0900441,0.0931049,0.148815,-0.0311285,0.0873161,-0.013034,0.197279
0.0366201,0.0247941,-0.0074406,-0.130307,-0.00454542,0.0706722,0.0580851,0.138726,-0.0612746,0.0100098,0.0926442,-0.109872,0.188528,0.070107,-0.243018,-0.0883422,-0.161722,-0.0430569,-0.108806,0.0445171,-0.0938514,0.0837879,-0.101003,0.0923395,0.0367333,0.170955,0.0769589,-0.0954282,0.0454559,0.0225037,-0.142307,0.13265,0.0190565,-0.0331045,-0.0554242,0.0254117,-0.000773486,0.0218308,-0.072466,-0.234315,0.0475351,-0.0336074,-0.0483978,-0.070312,0.0852649,-0.0661743,0.00752302,-0.0380539,0.0101574,-0.117551,0.0856244,-0.0168015,0.0140

In [0]:
# Add aggregated word embeddings 
ext_reviews = reviews.cbind(review_vecs)
# rename may be... all 'reviews' to 'news'
# ext becaus ewe are extending the reviews df with the vectors

In [0]:
ext_reviews["label_f"]  = ext_reviews["label"] .asfactor()

In [0]:
# Splitting to test train.. along with the review vecs is difficult without the 'Train' boolean column
# hence redoing it from that stage..
# so that we'll be able to do this following cell

In [0]:
ext_train = ext_reviews[ext_reviews["Train"] == "True"]
ext_test = ext_reviews[ext_reviews["Train"] == "True"]

In [0]:
#review_vecs.names
# had a 100 cols
# may be more would give a better model ..?

In [0]:
predictors =  review_vecs.names

In [0]:
response = 'label_f'

In [0]:
from h2o.automl import H2OAutoML

In [0]:
x = ext_train.columns

In [0]:
# x= [x for x in ext_train.columns if x not in ['label', 'Train']]

In [0]:
print(x)

['label', 'text', 'Train', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'C100', 'label_f']


In [0]:
# Identify predictors and response
x = ext_train.columns
x = [x for x in ext_train.columns if x not in ['label', 'Train']] # remove columns that can leak data
y = "label_f"
# other_cols = ""
# x.remove(y)

# For binary classification, response should be a factor
#train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()
# Already done

In [84]:
%%time
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)#, max_runtime_secs =1200)
aml.train(x=x, y=y, training_frame=ext_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 41.4 s, sys: 3.24 s, total: 44.7 s
Wall time: 1h 58min 49s


In [85]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20191024_103322,0.324476,1.15571,0.595409,0.354511
StackedEnsemble_AllModels_AutoML_20191024_103322,0.324684,1.1683,0.597048,0.356466
DRF_1_AutoML_20191024_103322,0.344187,1.82987,0.677304,0.458741
GBM_1_AutoML_20191024_103322,0.346206,1.3771,0.60089,0.361069
GBM_2_AutoML_20191024_103322,0.352272,1.38817,0.607581,0.369155
GBM_3_AutoML_20191024_103322,0.357094,1.43522,0.61599,0.379444
GBM_4_AutoML_20191024_103322,0.365243,1.50979,0.622982,0.388106
GBM_5_AutoML_20191024_103322,0.381277,1.51495,0.704841,0.496801
DeepLearning_1_AutoML_20191024_103322,0.414469,1.47603,0.657731,0.43261
GLM_grid_1_AutoML_20191024_103322_model_1,0.460027,2.47755,0.909008,0.826295


In [86]:
#aml.leader.confusion_matrix()
aml.leader.cross_validation_metrics_summary

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20191024_103322

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.03958210109769638
RMSE: 0.1989525096541795

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.3545113889522778
RMSE: 0.5954085899214738


<bound method ModelBase.cross_validation_metrics_summary of >

In [87]:
aml.leader.confusion_matrix(ext_test) # 


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,115,460,Error,Rate
0,380.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025641,10 / 390
1,0.0,446.0,1.0,0.0,0.0,0.0,0.0,12.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036717,17 / 463
2,0.0,1.0,460.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,20 / 480
3,0.0,0.0,1.0,463.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029350,14 / 477
4,0.0,0.0,0.0,0.0,434.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044053,20 / 454
5,0.0,0.0,0.0,0.0,0.0,465.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006410,3 / 468
6,0.0,0.0,0.0,0.0,0.0,0.0,453.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013072,6 / 459
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,472.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002114,1 / 473
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12.0,467.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027083,13 / 480
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,456.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046025,22 / 478



See the whole table with table.as_data_frame()


In [0]:
con_mat = aml.leader.confusion_matrix(ext_test).as_data_frame()

In [97]:
type(con_mat)

pandas.core.frame.DataFrame

In [100]:
con_mat.index # 23..?

RangeIndex(start=0, stop=23, step=1)

In [103]:
con_mat.iloc[19][19] #

277.0

In [104]:
con_mat.iloc[19][22] #

0.048109965635738834

In [115]:
con_mat.iloc[0:20,0:20]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,380.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,446.0,1.0,0.0,0.0,0.0,0.0,12.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,460.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,463.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,434.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,465.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,453.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,472.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12.0,467.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,456.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
np.sum(np.diag(con_mat.iloc[0:20,0:20]))/(con_mat.iloc[0:20,0:20].values.sum()) # sum of diagnoal elements/ sum of all in the matrix # 38733477 & SO 53762743

0.9728236853733981

In [0]:
# getting the absolute accuracy  as above

In [0]:
con_mat.to_csv('confusion_matrix.csv')

In [96]:
aml.leader.hit_ratio_table().as_data_frame() # SO 53762743

,k,hit_ratio
0,1,0.972824
1,2,0.975475
2,3,0.977684
3,4,0.980004
4,5,0.981330
5,6,0.983761
6,7,0.985086
7,8,0.986412
8,9,0.987738
9,10,0.988953
